In [ ]:
!pip install rank_bm25

import numpy as np
import pandas as pd
import re
from transformers import pipeline
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')  

from tqdm.notebook import tqdm
import pandas as pd
# test_dict_answer
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re
from huggingface_hub import login
import numpy as np
from sklearn.model_selection import train_test_split

login(token='')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
# status['claim']#     example = "Phát biểu tại phiên thảo luận về tình hình kinh tế xã hội của Quốc hội sáng 28/10 , Bộ trưởng Bộ LĐ-TB&XH Đào Ngọc Dung khái quát , tại phiên khai mạc kỳ họp , lãnh đạo chính phủ đã báo cáo , đề cập tương đối rõ ràng về việc thực hiện các chính sách an sinh xã hội"
tokenizer_seg = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation")
model_seg = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")
nlp = pipeline("token-classification", model=model_seg, tokenizer=tokenizer_seg)

def segmentation_token(example):
#     example = preprocess_text(example)
    ner_results = nlp(example)
    example_tok = ""
    for e in ner_results:
        if "##" in e["word"]:
            example_tok = example_tok + e["word"].replace("##","")
        elif e["entity"] =="I":
            example_tok = example_tok + "_" + e["word"]
        else:
            example_tok = example_tok + " " + e["word"]
    return example_tok

In [3]:
meta = pd.read_json('/kaggle/input/ise-dsc01/ise-dsc01-train.json').T
meta.head(3)

,context,claim,verdict,evidence,domain
7125,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...",giao-duc
7126,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...",giao-duc
7127,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Nhằm phát triển đội ngũ, FABA thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...",giao-duc


Không phải cứ “ là chấm được có những câu " chuaq phải là chấm
các dữ liệu khi đưa vào proces của BTC thì nó sẽ đưa về dạng chuẩn hết nên ko phải lo"


In [4]:
meta['id'] = meta.index

In [5]:
meta = meta.drop_duplicates()

In [6]:
count_value = meta.claim.value_counts().values
value = meta.claim.value_counts().keys()

In [7]:
cnt = 0
wrong_claims = []
for i in range(len(count_value)):
    if count_value[i] > 1:
        cnt+=1
        wrong_claims.append(value[i])

In [8]:
# meta[meta.claim.isin(wrong_claims)]

In [9]:
idx_list = meta[meta.claim.isin(wrong_claims)].sort_values(by='claim').index

In [10]:
meta.drop(idx_list, inplace=True)

In [11]:
 meta[meta.claim.isin(wrong_claims)].sort_values(by='claim')

,context,claim,verdict,evidence,domain,id


In [12]:
id_n = []
for i in meta.index:
    if meta.verdict[i] == 'NEI': continue
    if "\n\n" in meta.evidence[i]:
        id_n.append(i)

In [13]:
len(meta)

36482

In [14]:
meta.drop(id_n, inplace=True)

In [15]:
id_wrong = []
for i in meta.index:
    if meta.evidence[i] != None:
        if meta.claim[i] in meta.context[i] and meta.claim[i] not in  meta.evidence[i]:
            id_wrong.append(i)

In [16]:
meta['id'] = meta.index

In [17]:
# process del id has claim equal evidence in Refuted
meta.drop(meta[(meta.claim == meta.evidence) & (meta.verdict == 'REFUTED')].index, inplace=True)

In [18]:
# meta[meta.claim == meta.evidence].verdict.value_counts()

In [19]:
meta.drop(meta[meta.claim == meta.evidence].index, inplace= True)

In [20]:
len(meta)

35849

In [21]:
evi = []
for i in meta.index:
    if meta.verdict[i] == 'NEI':
        evi.append("")
    else:
        evi.append(meta.evidence[i])

In [22]:
meta.evidence = evi

In [25]:
# preprocess_text('\n\nỐc bươu nướng tiêu xanh\n\nĐây là một')

In [26]:
def process_data(text):
    
    # Process del \n\n
    text = re.sub(r'(\n)+', ' ',text) 
    # 
#     text = re.sub(r'\.{2,}\s*([a-z])', r', \1',  text)

#     text = re.sub(r'\.{2,}\s*([A-Z])', r'. \1',  text)

#     text = re.sub(r'([0-9]{1}).([0-9]{1})', r'\1\2',  text)
    text = re.sub(r'(\d)\.(\d)', r'\1\2', text)
#     return text
    text = text.replace('TP. HCM', 'TP.HCM')
#     line_text = text.split('. ')
#     new_line = []
#     for line  in line_text:
#         line = line.strip()
#         if line == '.' or line == '' or line == ' ':
# #             print('.')
#             continue
# #         line = preprocess_text(line)
# #         print(line)
#         new_line.append(line)
    return text

In [27]:
process_data('Đại diện Khu công nghệ cao Hòa Lạc đề xuất, SHTP cần nâng cao năng lực của nhân sự các tổ chức trung gian có chức năng R&D, đào tạo, ươm tạo..')

'Đại diện Khu công nghệ cao Hòa Lạc đề xuất, SHTP cần nâng cao năng lực của nhân sự các tổ chức trung gian có chức năng R&D, đào tạo, ươm tạo..'

In [28]:
meta.context = meta.context.apply(lambda x: process_data(x))

In [29]:
meta.claim = meta.claim.apply(lambda x: process_data(x))

In [30]:
meta.evidence = meta.evidence.apply(lambda x: process_data(x) if x != None else '')

In [36]:
from pyvi import ViTokenizer, ViPosTagger

ViTokenizer.tokenize(u"Trường đại học bách khoa hà nội")

'Trường đại_học bách_khoa hà_nội'

In [37]:
# !pip install pyvi

In [38]:
id_del_eviden_than2 = []

for i in meta.index:
    if meta.verdict[i] == 'NEI': continue
    if len(ViTokenizer.tokenize(meta.evidence[i]).split()) <= 3:
        id_del_eviden_than2.append(i)

In [39]:
meta.drop(id_del_eviden_than2, inplace=True)
print(f"Have {len(id_del_eviden_than2)} case <= 2 ")

Have 5 case <= 2 


In [40]:
meta.verdict.value_counts()

verdict
REFUTED      12350
NEI          12276
SUPPORTED    11212
Name: count, dtype: int64

In [42]:
def preprocess_text(text: str) -> str:    
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text

In [43]:
## check evidence not in context line
id_not_line = []
for i in meta[meta.verdict != 'NEI'].index:
    context = meta.context[i].split(". ")
    CHECK = False
    for line in context:
        if preprocess_text(line) == preprocess_text(meta.evidence[i]):
            CHECK=True
            break
    if CHECK == False:
        id_not_line.append(i)

In [44]:
meta[meta.id.isin(id_not_line)].verdict.value_counts()

verdict
REFUTED      264
SUPPORTED     32
Name: count, dtype: int64

In [45]:
meta.drop(id_not_line, inplace = True)

In [48]:
meta['id'] = meta.index

In [49]:
idx_not_find_evi = []
for i in tqdm(meta.index):
    if meta.verdict[i] != 'NEI':
#         context = meta.context[i].split('. ')
        idx = meta.context[i].find(meta.evidence[i])
        if idx ==-1:
            idx_not_find_evi.append(i)
            print('==========================')
            print(meta.context[i])
            print(meta.evidence[i])
            
print('idx_not_find_evi')
print(idx_not_find_evi)
meta.drop(idx_not_find_evi, inplace=True)

  0%|          | 0/35542 [00:00<?, ?it/s]

Festival biển Nha Trang 2023 với chủ đề chủ đề "Khánh Hòa - Khát vọng phát triển" lần thứ 10 vừa diễn ra đã đón hàng trăm nghìn lượt khách, thu về gần 550 tỷ đồng. Festival diễn ra ngày 3-6/6, địa phương đã tổ chức chuỗi hoạt động đa dạng về nội dung và hình thức, phản ánh được những giá trị văn hóa của con người, tinh thần lao động hăng say, tình yêu biển đảo quê hương. Chương trình đã tạo được tiếng vang, được sự quan tâm của người dân, du khách trong và ngoài nước. Độ cao bay tối đa đạt 250 m và bán kính hoạt động 250 m. Mở đầu là hình ảnh chim yến vút bay lên bầu trời, thể hiện sự vươn cao, vươn xa của địa phương, khi các drone trình diễn ánh sáng đã được lập trình qua máy tính từ những hình ảnh sắp xếp hay đường bay, quỹ đạo bay. Cùng gia đình ở TP HCM tới Nha Trang chơi, chị Nguyễn Thị Phương, 35 tuổi, đã tới quảng trường xem khai mạc. “Mình nhiều lần xem Festival biển, song lần này khá ấn tượng với các tiết mục trong dịp lễ này, đặc biệt là bữa tiệc vẽ tranh ánh sáng trên bầu tr

In [50]:
meta.verdict.value_counts()

verdict
NEI          12276
REFUTED      12086
SUPPORTED    11179
Name: count, dtype: int64

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def compute_tfidf(context, claim, model = False, thres= 0.5):
    
    tfidf_vectorizer = TfidfVectorizer()
    corpus = context.split('. ') 
    answer = corpus.copy()
    
#     corpus.append(claim)
    claim = segmentation_token(claim)
    len_claim = len(claim.split(' '))
    corpus_pro = []
    for i in range(len(corpus)):
        
        corpus[i] = segmentation_token(corpus[i])
        
        sentence = corpus[i]
        
#         l = len(sentence.split(' '))
        
#         p  = l/len_claim
        
#         if i != 0 and p < thres and l > 1 :
#             sentence = f'{corpus[i-1]} . {sentence}'
        corpus_pro.append(sentence)
    corpus_pro.append(claim)
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus_pro)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    idx = np.argmax(cosine_sim[len(corpus_pro)-1, :-1])
    return answer[idx]

In [52]:
def get_top_evidence(claim, context):
    claim_value = claim
    context_value = context

    context = context_value.split('. ')
    context = [line for line in context]

    tokenized_context = [doc.split(' ') for doc in context]
    bm25 = BM25Okapi(tokenized_context)
    scores = bm25.get_scores(claim_value.split())

    score_sentence_pairs = sorted(zip(scores, context), reverse=True)
    highest_sentence = []
    for _, x in score_sentence_pairs[:25]:
        highest_sentence.append(x)
#     _, highest_sentence = score_sentence_pairs[0]
    return highest_sentence

In [54]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util

ModuleNotFoundError: No module named 'sentence_transformers'

In [232]:
model_semantic_search = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
model_semantic_search.to('cuda')
def compute_simulate(sentence1, sentence2):
    #Compute embedding for both lists
    
    embedding_1= model_semantic_search.encode(sentence1, convert_to_tensor=True, show_progress_bar=False, device='cuda')
    embedding_2 = model_semantic_search.encode(sentence2, convert_to_tensor=True, show_progress_bar=False, device='cuda')

    return util.pytorch_cos_sim(embedding_1, embedding_2).item()

In [55]:
def get_top_evidence(claim, context, evidence):
    claim_value = claim
    context_value = context

    context = context_value.split('. ')
    context = [line for line in context]

    tokenized_context = [doc.split(' ') for doc in context]
    bm25 = BM25Okapi(tokenized_context)
    scores = bm25.get_scores(claim_value.split())

    score_sentence_pairs = sorted(zip(scores, context), reverse=True)
    highest_sentence = []
    
    for _, x in score_sentence_pairs[:1]:
        highest_sentence.append(x)
#     if evidence != "":
#         if evidence not in highest_sentence:
#             highest_sentence.append(evidence)
#     _, highest_sentence = score_sentence_pairs[0]
    return highest_sentence[0]

In [173]:
data = meta.copy()

In [175]:
new_data = []

for i in tqdm(data.index):
    
    lines = data.context[i].split('.')
    tokens = data.context[i].split(' ')
    if len(tokens) <= 400: 
        new_data.append({
                'id': data['id'][i],
                'context': data['context'][i],
                'claim': data['claim'][i],
                'verdict': data['verdict'][i],
                'evidence': data['evidence'][i]
            })
        continue
        
    token_line = [l.split(' ') for l in lines]
    
    tmp_context_token = []
    tmp_context = []
    cnt = 0
    error = []
    for idx in range(len(lines)):
        check = True
        if len(token_line[idx] + tmp_context_token) <=400:
            tmp_context_token += token_line[idx]
            tmp_context.append(lines[idx])
            check = False
        
        if len(token_line[idx] + tmp_context_token) > 400 or idx == len(lines) - 1:
            context = '.'.join(tmp_context)
            if len(context) == 0: continue
            label = 'NEI'
            evidence = ''
            if context.find(data['evidence'][i]) != -1 and data['verdict'][i] != 'NEI':
                label = data['verdict'][i]
                evidence = data['evidence'][i]
                cnt+=1
                error.append({
                    'id': data['id'][i],
                    'context':context,
                    'evidence': data['evidence'][i],
                    'label':label,
                    'claim': data['claim'][i]
                })
            new_data.append({
                'id': data['id'][i],
                'context': context,
                'claim': data['claim'][i],
                'verdict': label,
                'evidence': evidence
            })
            if check:
                # Reset status
                tmp_context_token = token_line[idx] 
                tmp_context = [lines[idx]]
            else:
                tmp_context_token = []
                tmp_context = []
#     print(cnt)
    if cnt>1:
        print(error)
        print("++++++++++++++")

  0%|          | 0/35541 [00:00<?, ?it/s]

[{'id': 34515, 'context': 'Đại nội Huế nằm trong Kinh thành Huế được xây dựng năm 1804 dưới thời vua Gia Long, đến năm 1833 đời vua Minh Mạng mới hoàn thiện với khoảng 147 công trình. Toàn bộ hệ thống cung điện được bố trí trên một trục đối xứng, trong đó trục chính giữa là các công trình chỉ dành cho vua. Các công trình ở hai bên được phân bố chặt chẽ theo từng khu vực, tuân thủ nguyên tắc "tả nam hữu nữ", "tả văn hữu võ". Khu vực Đại nội nổi bật với các công trình và hiện vật như cửa Ngọ Môn, lầu Ngũ Phụng, điện Thái Hòa nơi đặt ngai vàng, Thế Tổ Miếu nơi đặt cửu đỉnh hay điện Kiến Trung đang được trung tu, tu bổ. Giá vé tham quan 200000 đồng. Lăng vua Gia Long hay Thiên Thọ lăng ở xã Hương Thọ là nơi an nghỉ của hoàng đế Gia Long (1762- 1820) cùng hai người vợ là Thừa Thiên Cao Hoàng hậu và Thuận Thiên Cao Hoàng hậu. Lăng được xây dựng từ năm 1814 đến năm 1820. Quần thể lăng vua Gia Long có vị trí phong thủy đẹp nhất trong những lăng vua triều Nguyễn. Sử cũ cho hay, thầy địa lý Lê D

In [176]:
meta.verdict.value_counts()

verdict
NEI          12276
REFUTED      12086
SUPPORTED    11179
Name: count, dtype: int64

In [177]:
data = pd.DataFrame(new_data)
data.verdict.value_counts()

verdict
NEI          55135
REFUTED      11768
SUPPORTED    10928
Name: count, dtype: int64

In [179]:
data['idx'] = range(len(data))

In [180]:
data = data.sample(frac=1)

In [200]:
id_list_support = np.unique(meta[meta.verdict=='SUPPORTED'].id)
id_list_refuse = np.unique(meta[meta.verdict=='REFUTED'].id)
id_list_nei = np.unique(meta[meta.verdict=='NEI'].id)

print(len(id_list_support), len(id_list_refuse), len(id_list_nei))

11179 12086 12276


In [201]:
train_support, test_support  = train_test_split(id_list_support, test_size=0.2, random_state=42)
train_refuse, test_refuse  = train_test_split(id_list_refuse, test_size=0.2, random_state=42)
train_nei, test_nei  = train_test_split(id_list_nei, test_size=0.5, random_state=42)

In [202]:
train_support = data[data.id.isin(train_support)]
train_refuse = data[data.id.isin(train_refuse)]
train_nei = data[data.id.isin(train_nei)]

In [203]:
test_support = data[data.id.isin(test_support)]
test_refuse = data[data.id.isin(test_refuse)]
test_nei = data[data.id.isin(test_nei)]
test_nei = test_nei.sample(n  =2500)

In [209]:
train = pd.concat([train_support, train_refuse, train_nei]).sample(frac=1, random_state=42)
test = pd.concat([test_support, test_refuse, test_nei]).sample(frac=1, random_state=42)

In [210]:
len(np.unique(train.id)), len(np.unique(test.id))


(24749, 6797)

In [211]:
train.verdict.value_counts()


verdict
NEI          35934
REFUTED       9410
SUPPORTED     8741
Name: count, dtype: int64

In [212]:
test.verdict.value_counts()

verdict
NEI          8051
REFUTED      2358
SUPPORTED    2187
Name: count, dtype: int64

In [213]:
train.to_csv('/kaggle/working/train.csv', escapechar='\\')

In [214]:
test.to_csv('/kaggle/working/test.csv', escapechar='\\')

In [216]:
description = '''
Data had follow weight trained with data old version 21

Train: 
    NEI          35934
    REFUTED       9410
    SUPPORTED     8741
Test:
    NEI          8051
    REFUTED      2358
    SUPPORTED    2187
'''
# Define the file path where you want to write the description
file_path = "description.txt"

# Open the file in write mode and write the description to it
with open(file_path, 'w') as file:
    file.write(description)

print(f'Description has been written to {file_path}')

Description has been written to description.txt
